In [1]:
%%capture

!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install accelerate -U
!pip install -U transformers

In [2]:
import pandas as pd
from datasets import Dataset, load_dataset, DatasetDict
import os
import librosa
import numpy as np
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from datasets import Audio
from collections import OrderedDict
import evaluate
from transformers import WhisperForConditionalGeneration
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 1695884081.2863135
 1695884081.3142548
 1695884126.3520193
 1695884126.3613842
 annotaion_person_draw0.zip
 annotaion_person_draw.zip
 Audio2Head
 chip1.py
 coco_person_car_yolov4_22000.weights
'Colab Notebooks'
 config.json
'Copy of Copy of Copy of yolov4-custom_TRAINING_vf.ipynb'
'Copy of Copy of custom_yoloV5.ipynb'
'Copy of Copy of productCount_final_custom_yolov3.ipynb'
 events.out.tfevents.1695884081.a48e7dd1b69d.18859.0
 events.out.tfevents.1695884126.a48e7dd1b69d.19062.0
 Face_Recognition
 fake_review
'Fraser (1).docx'
 Fraser.gdoc
 GAN
'Gan_model '
 indian_test_team.jpg
 kaggle.json
'Latest Screenshots.docx'
 LipGAN
'model (1).safetensors'
 model.py
 model.safetensors
'My Drive'
 new_annotation
 preprocessor_config.json
 pretrained_models.zip
'Proposal for VA - FT.docx'
'Proposal 

In [6]:
#!tar -xf /content/gdrive/MyDrive/speech-text_custom_train_all_pretrained/fine_tune_whisper_v2/test-clean.tar.gz -C /content/librispeech_asr_dataset


In [4]:
#!pip install torchaudio datasets pydub
from datasets import load_dataset
#load daatest locallyy folder
#dataset = load_dataset("audiofolder", data_dir="/content/librispeech_asr_dataset/LibriSpeech/test-clean",  drop_labels=True)
dataset= load_dataset("CristianaLazar/librispeech5k_train")

dataset

DatasetDict({
    train.360: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id', 'input_features', 'labels'],
        num_rows: 5000
    })
})

In [5]:
dataset["train"] = dataset.pop("train.360")

dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id', 'input_features', 'labels'],
        num_rows: 5000
    })
})

In [6]:
dataset['train']['audio'][1:10]

[{'path': None,
  'array': array([-0.00042725, -0.00039673, -0.00039673, ..., -0.00210571,
         -0.00216675, -0.00204468]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([-1.98364258e-03, -2.04467773e-03, -1.95312500e-03, ...,
          6.10351562e-05,  9.15527344e-05,  1.22070312e-04]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([ 0.00012207,  0.00012207,  0.00012207, ..., -0.00772095,
         -0.00784302, -0.0067749 ]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([-6.34765625e-03, -5.85937500e-03, -5.49316406e-03, ...,
          6.10351562e-05,  1.83105469e-04, -1.22070312e-04]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([-3.66210938e-04, -7.01904297e-04, -1.09863281e-03, ...,
          6.10351562e-05,  6.10351562e-05,  3.05175781e-05]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([ 0.00000000e+00,  6.10351562e-05,  0.00000000e+00, ...,
         -1.12915039e-03, -1.25122070e-03, -1.28173828e-03]),
 

In [ ]:
'''
import os
import librosa
import numpy as np
from tqdm import tqdm

# Replace 'path_to_test_clean_dataset' with the actual path to your downloaded dataset
dataset_path = '/content/test-clean'

# Create empty lists to store audio data and corresponding transcriptions
audio_data = []
transcriptions = []

# Iterate through the directories in the dataset path
for speaker_folder in tqdm(os.listdir(dataset_path), desc="Processing Speakers", unit="speaker"):
    speaker_path = os.path.join(dataset_path, speaker_folder)

    # Check if the item in the directory is a folder
    if os.path.isdir(speaker_path):
        # Iterate through the subdirectories (containing audio files) in the speaker folder
        for subfolder in os.listdir(speaker_path):
            subfolder_path = os.path.join(speaker_path, subfolder)

            # Check if the item in the subdirectory is a folder
            if os.path.isdir(subfolder_path):
                # Find audio files in the subdirectory
                audio_files = [file for file in os.listdir(subfolder_path) if file.endswith('.flac')]

                # Assuming there is only one transcription file in the subdirectory
                transcription_file = [file for file in os.listdir(subfolder_path) if file.endswith('.trans.txt')][0]
                transcription_file_path = os.path.join(subfolder_path, transcription_file)

                # Read transcription
                with open(transcription_file_path, 'r') as transcription_file:
                    transcription = transcription_file.read().strip()

                # Process each audio file in the subdirectory
                for audio_file in audio_files:
                    audio_file_path = os.path.join(subfolder_path, audio_file)

                    # Load audio file
                    audio, _ = librosa.load(audio_file_path, sr=None)

                    # Append audio data and transcription to lists
                    audio_data.append(audio)
                    transcriptions.append(transcription)

# Convert lists to numpy arrays if needed
audio_data = np.array(audio_data)
transcriptions = np.array(transcriptions)

# Now you can use 'audio_data' and 'transcriptions' in your machine learning pipeline
'''

In [ ]:
#dataset = load_dataset("audiofolder", data_dir="./audio_folder/", split="train")
#dataset= load_dataset("mozilla-foundation/common_voice_11_0", "en", split="test")
#dataset= load_dataset("librispeech_asr", split="validation.clean")
#dataset= load_dataset("/content/librispeech_asr_dataset/LibriSpeech/test-clean")
#dataset = load_dataset("./test-clean/", split="train")
#train_test = dataset.train_test_split(test_size=0.22)
#common_voice = DatasetDict(train_test)
#print(common_voice)


In [ ]:
#dataset = dataset.shuffle(seed=42).select([i for i in range(1000)])
#dataset

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 1000
})

In [9]:
train_test = dataset['train'].train_test_split(test_size=0.22)
common_voice = DatasetDict(train_test)
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id', 'input_features', 'labels'],
        num_rows: 3900
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id', 'input_features', 'labels'],
        num_rows: 1100
    })
})


In [ ]:
for i in range(len(common_voice["train"])):
    print("Audio: ",common_voice["train"][i]['audio']['path'])
    print("Sentence: ",common_voice["train"][i]['sentence'])
print("\n\n")
for i in range(len(common_voice["test"])):
     print("Audio: ",common_voice["test"][i]['audio']['path'])
     print("Sentence: ",common_voice["test"][i]['sentence'])

In [10]:
#common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
common_voice = common_voice.remove_columns(["file",  'speaker_id', 'chapter_id', 'id', 'input_features', 'labels'])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 3900
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 1100
    })
})


In [11]:
# Rename the 'text' column to 'sentence' in the train split
common_voice["train"] = common_voice["train"].rename_column("text", "sentence")

# Rename the 'text' column to 'sentence' in the test split
common_voice["test"] = common_voice["test"].rename_column("text", "sentence")
common_voice

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 3900
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1100
    })
})

In [12]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="english", task="transcribe")
tokenizer.save_pretrained("tokenizer/")
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="english", task="transcribe")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [13]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out s>pecial: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 YET IF YOU WERE TO ASK THE QUESTION YET IT IS INSTRUCTIVE AND INTERESTING YET IT IS NO LESS TRUE YET IT IS PERFECTLY PLAIN YET
Decoded w/ special:    <|startoftranscript|><|en|><|transcribe|><|notimestamps|>YET IF YOU WERE TO ASK THE QUESTION YET IT IS INSTRUCTIVE AND INTERESTING YET IT IS NO LESS TRUE YET IT IS PERFECTLY PLAIN YET<|endoftext|>
Decoded w/out s>pecial: YET IF YOU WERE TO ASK THE QUESTION YET IT IS INSTRUCTIVE AND INTERESTING YET IT IS NO LESS TRUE YET IT IS PERFECTLY PLAIN YET
Are equal:             True


In [14]:
print(common_voice["train"][0]['audio'])

{'path': None, 'array': array([ 0.02972412,  0.02975464,  0.02972412, ..., -0.0451355 ,
       -0.04449463, -0.04400635]), 'sampling_rate': 16000}


In [15]:
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [16]:
print(common_voice["train"][0])

{'audio': {'path': None, 'array': array([ 0.02972412,  0.02975464,  0.02972412, ..., -0.0451355 ,
       -0.04449463, -0.04400635]), 'sampling_rate': 16000}, 'sentence': 'YET IF YOU WERE TO ASK THE QUESTION YET IT IS INSTRUCTIVE AND INTERESTING YET IT IS NO LESS TRUE YET IT IS PERFECTLY PLAIN YET'}


In [17]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    print(audio)
    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [18]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"])

Map:   0%|          | 0/3900 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
{'path': None, 'array': array([ 7.32421875e-04,  1.86157227e-03,  2.74658203e-03, ...,
        2.16674805e-03,  7.93457031e-04, -6.10351562e-05]), 'sampling_rate': 16000}
{'path': None, 'array': array([ 3.05175781e-05, -9.15527344e-05,  1.22070312e-04, ...,
       -3.05175781e-05, -6.10351562e-05,  3.05175781e-05]), 'sampling_rate': 16000}
{'path': None, 'array': array([-0.00085449, -0.00131226, -0.00961304, ..., -0.00036621,
       -0.01480103, -0.00845337]), 'sampling_rate': 16000}
{'path': None, 'array': array([-0.00158691,  0.00582886,  0.00906372, ..., -0.00045776,
       -0.00274658, -0.00219727]), 'sampling_rate': 16000}
{'path': None, 'array': array([0.00610352, 0.00616455, 0.00558472, ..., 0.00531006, 0.00430298,
       0.00476074]), 'sampling_rate': 16000}
{'path': None, 'array': array([ 0.00076294,  0.00042725, -0.00030518, ...,  0.00021362,
        0.00030518,  0.00024414]), 'sampling_rate': 16000}
{'path': None, 'array': a

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

{'path': None, 'array': array([0.00128174, 0.00112915, 0.00335693, ..., 0.00402832, 0.00631714,
       0.00363159]), 'sampling_rate': 16000}
{'path': None, 'array': array([ 0.00024414,  0.00146484, -0.00042725, ..., -0.00170898,
       -0.00204468, -0.00045776]), 'sampling_rate': 16000}
{'path': None, 'array': array([ 0.00436401,  0.00439453,  0.00585938, ..., -0.00045776,
        0.00057983, -0.00088501]), 'sampling_rate': 16000}
{'path': None, 'array': array([-9.76562500e-04, -1.00708008e-03, -3.05175781e-05, ...,
       -6.10351562e-05, -1.22070312e-04, -5.18798828e-04]), 'sampling_rate': 16000}
{'path': None, 'array': array([0.05462646, 0.05435181, 0.05429077, ..., 0.05209351, 0.05215454,
       0.05230713]), 'sampling_rate': 16000}
{'path': None, 'array': array([-0.00152588, -0.00210571,  0.00244141, ..., -0.00021362,
        0.00015259,  0.00076294]), 'sampling_rate': 16000}
{'path': None, 'array': array([ 0.00201416,  0.00048828, -0.00085449, ..., -0.00027466,
       -0.00192261

In [19]:
common_voice

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3900
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1100
    })
})

In [20]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [21]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [22]:
metric = evaluate.load("wer")

In [23]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [24]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

In [25]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [27]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-base-pron",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=500,  # Set max_steps to 100 for 100 iterations
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,  # Save every 100 steps
    eval_steps=100,  # Evaluate every 100 steps
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)


In [37]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-base-pron",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=1000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)


In [38]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [39]:
import torch
torch.cuda.empty_cache()

In [40]:
from huggingface_hub import notebook_login

notebook_login()

In [41]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
500,0.251600,0.290967,51.457818


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,0.251600,0.290967,51.457818
1000,0.108100,0.233480,32.064102


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=1000, training_loss=0.339587260723114, metrics={'train_runtime': 3211.4938, 'train_samples_per_second': 4.982, 'train_steps_per_second': 0.311, 'total_flos': 1.03672217862144e+18, 'train_loss': 0.339587260723114, 'epoch': 4.1})

In [51]:
#trainer.save_model("/content/whisper_trained")

In [42]:
from transformers import pipeline
import gradio as gr

tokenizer = WhisperTokenizer.from_pretrained('./tokenizer/', language="english", task="transcribe")
# model = WhisperForConditionalGeneration.from_pretrained("./whisper-base-pron")

pipe = pipeline("automatic-speech-recognition", model="./whisper-base-pron", tokenizer=tokenizer)

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

print(pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from transformers import pipeline
import gradio as gr

tokenizer = WhisperTokenizer.from_pretrained('./tokenizer/', language="english", task="transcribe")
# model = WhisperForConditionalGeneration.from_pretrained("./whisper-base-pron")

#pipe = pipeline("automatic-speech-recognition", model="./whisper-base-pron", tokenizer=tokenizer)
pipe = pipeline("automatic-speech-recognition", model="/content/whisper-base-pron/checkpoint-1000", tokenizer=tokenizer)

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

# Test on the given audio file
audio_path = "/content/2086-149220-0033.wav"  # Replace with the actual path
result = transcribe(audio_path)
print(result)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 WELL I DON'T WISH TO SEE IT ANY MORE OBSERVE FIBI TURNING AWAY HER EYES IT IS CERTAINLY VERY LIKE THE OLD PORTRAIT


In [9]:
!wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav

--2023-12-01 14:25:24--  https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 3.5.129.2, 16.12.65.66, 52.219.177.226, ...
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|3.5.129.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237964 (232K) [audio/wav]
Saving to: ‘2086-149220-0033.wav.1’

2086-149220-0033.wa 100%[===================>] 232.39K  --.-KB/s    in 0.1s    

2023-12-01 14:25:24 (1.87 MB/s) - ‘2086-149220-0033.wav.1’ saved [237964/237964]



In [43]:
'''
iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="Whisper Small Pronunciation",
    description="Realtime demo for English speech recognition using a fine-tuned Whisper base model.",
)

iface.launch()
'''

TypeError: ignored

In [ ]:
#upload to hgugging face

In [ ]:
kwargs = {
    "dataset_tags": "CristianaLazar/librispeech5k_train",
    "dataset": "librispeech5k_train",  # a 'pretty' name for the training dataset
    "language": "en",
    "model_name": "openai/whisper-base - Manish Kumar",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-base",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
#The training results can now be uploaded to the Hub. To do so, execute the push_to_hub command:

trainer.push_to_hub(**kwargs)

In [6]:
!rm -rf /content/whisper-base-pron/checkpoint-500

In [8]:
!cp -r /content/tokenizer /content/gdrive/MyDrive/speech-text_custom_train_all_pretrained/fine_tune_whisper_v2